In [1]:
import cv2
import os
import glob
import warnings
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import itertools
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D, MaxPooling2D, Flatten,BatchNormalization,Dropout,Input
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import HeNormal
import keras_tuner as kt
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.initializers import HeNormal
import gc

In [ ]:
styles = [
    'Achaemenid architecture',
    'American craftsman style',
    'American Foursquare architecture',
    'Ancient Egyptian architecture',
    'Art Deco architecture',
    'Art Nouveau architecture',
    'Baroque architecture',
    'Bauhaus architecture',
    'Beaux-Arts architecture',
    'Byzantine architecture',
    'Chicago school architecture',
    'Colonial architecture',
    'Deconstructivism',
    'Edwardian architecture',
    'Georgian architecture',
    'Gothic architecture',
    'Greek Revival architecture',
    'International style',
    'Novelty architecture',
    'Palladian architecture',
    'Postmodern architecture',
    'Queen Anne architecture',
    'Romanesque architecture',
    'Russian Revival architecture',
    'Tudor Revival architecture'
] 
file_jpg = glob.glob(f'/kaggle/input/architectural-styles-dataset/**/{styles[5]}/*.jpg', recursive=True)

dim = (256,256)

for i, f in enumerate(file_jpg[:5]):
    img = cv2.imread(f)
    img = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    plt.imshow(img)
    plt.show()

In [ ]:
# Canny edges to eliminate noise 

X = []
y = []

dim = (256,256)

for style in styles[:4]:
    file_jpg = glob.glob(f'/kaggle/input/architectural-styles-dataset/**/{style}/*.jpg', recursive=True)

    for i, f in enumerate(file_jpg):
        img = cv2.imread(f)
        img = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = np.array(img)
        img = img.astype('float32')
        img /= 255 
        X.append(img)
        y.append(style)

In [ ]:
y = [styles.index(label) for label in y]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [5]:
from sklearn.preprocessing import LabelEncoder
y_train = LabelEncoder().fit_transform(y_train)
y_test = LabelEncoder().fit_transform(y_test)

In [6]:
len(styles)

25

In [7]:
X_train = np.expand_dims(X_train, axis=-1)  # Add channel dimension
X_train = np.repeat(X_train, 3, axis=-1)   # Repeat for RGB
X_test = np.expand_dims(X_test, axis=-1)
X_test = np.repeat(X_test, 3, axis=-1)

In [8]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), padding='valid', activation='relu', 
                 input_shape=(256, 256, 3),
                 kernel_initializer=HeNormal()))  # He Normal Initialization
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'))

model.add(Conv2D(64, kernel_size=(3, 3), padding='valid', activation='relu', 
                 kernel_initializer=HeNormal()))  
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'))

model.add(Conv2D(128, kernel_size=(3, 3), padding='valid', activation='relu', 
                 kernel_initializer=HeNormal()))  
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'))

model.add(Flatten())

model.add(Dense(128, activation='relu',
                kernel_initializer=HeNormal()))  
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu',
                kernel_initializer=HeNormal())) 
model.add(Dropout(0.2))
model.add(Dense(25, activation='softmax', kernel_initializer=HeNormal()))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 254, 254, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 254, 254, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 127, 127, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 125, 125, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 125, 125, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 62, 62, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 60, 60, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 60, 60, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 30, 30, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 115200)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      14,745,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 25)                  │           1,625 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,849,753 (56.65 MB)

 Trainable params: 14,849,305 (56.65 MB)

 Non-trainable params: 448 (1.75 KB)

In [10]:
model.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=["accuracy"])
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [ ]:
history = model.fit(X_train,y_train,epochs =100,validation_data=(X_test,y_test),callbacks=[early_stopping])

Epoch 1/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 135s 4s/step - accuracy: 0.2870 - loss: 18.3682 - val_accuracy: 0.3082 - val_loss: 6.9929
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 131s 4s/step - accuracy: 0.3708 - loss: 7.3474 - val_accuracy: 0.2688 - val_loss: 19.5110
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 132s 4s/step - accuracy: 0.4449 - loss: 3.5092 - val_accuracy: 0.2796 - val_loss: 11.0410
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 131s 4s/step - accuracy: 0.4847 - loss: 2.5539 - val_accuracy: 0.2760 - val_loss: 9.0929
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 143s 4s/step - accuracy: 0.4669 - loss: 2.2852 - val_accuracy: 0.3584 - val_loss: 6.7987
Epoch 6/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 136s 4s/step - accuracy: 0.5552 - loss: 1.7198 - val_accuracy: 0.3835 - val_loss: 4.4207
Epoch 7/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 132s 4s/step - accuracy: 0.5155 - loss: 1.8697 - val_accuracy: 0.4265 - val_loss: 3.0226
Epoch 8/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 132s 4s/step - accuracy: 0.5691 - loss: 1.4947 - val_accuracy:

In [ ]:
import matplotlib.pyplot as plt
history = history.history
# Plot 1: Training and Validation Loss
plt.subplot(1, 2, 1)  
plt.plot(history['val_loss'], color='red', label='Validation Loss')
plt.plot(history['loss'], color='blue', label='Training Loss')
plt.title('Validation and Training Loss ')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot 2: Training and Validation Accuracy
plt.subplot(1, 2, 2) 
plt.plot(history['val_accuracy'], color='red', label='Validation Accuracy')
plt.plot(history['accuracy'], color='blue', label='Training Accuracy')
plt.title('Validation and Training Accuracy ')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Adjust layout for better spacing
plt.tight_layout(pad=50.0)
plt.show()

In [ ]:


# Model-building function with Hyperparameters (using Keras Tuner)
def build_model(hp):
    model = Sequential()
    model.add(Input(shape=(256, 256, 3)))
    
    # Convolutional layers
    for i in range(hp.Int("num_conv_layers", 1, 4)):  # 1 to 4 Conv layers
        model.add(
            Conv2D(
                filters=hp.Choice(f"filters_{i}", [32, 64, 128, 256]),
                kernel_size=hp.Choice(f"kernel_size_{i}", [3, 5]),
                activation="relu",
                padding="valid",
                kernel_initializer=HeNormal(),
            )
        )
        if hp.Boolean(f"batch_norm_{i}"):
            model.add(BatchNormalization())  # Add Batch Normalization conditionally
        model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding="valid"))
    
    model.add(Flatten())
    
    # Dense layers
    for j in range(hp.Int("num_dense_layers", 1, 3)):  # 1 to 3 Dense layers
        model.add(
            Dense(
                units=hp.Int(f"units_dense_{j}", min_value=64, max_value=256, step=64),
                activation="relu",
                kernel_initializer=HeNormal(),
            )
        )
        model.add(Dropout(hp.Float(f"dropout_dense_{j}", 0.2, 0.5, step=0.1)))  # Tune dropout rate
    
    model.add(Dense(25, activation='softmax', kernel_initializer=HeNormal()))

    # Optimizer setup
    optimizer = hp.Choice("optimizer", ["adam", "sgd", "rmsprop"])
    learning_rate = hp.Float("learning_rate", 1e-4, 1e-2, sampling="log")
    
    if optimizer == "adam":
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == "sgd":
        opt = SGD(learning_rate=learning_rate, momentum=hp.Float("momentum", 0.5, 0.9))
    else:
        opt = RMSprop(learning_rate=learning_rate)

    # Compile model
    model.compile(optimizer=opt, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    
    return model

# Hyperparameter Tuning using Hyperband
tuner = kt.Hyperband(
    build_model,
    objective="val_accuracy",
    max_epochs=16,
    factor=3,
    directory="hyperband_tuning",
    project_name="cnn_hyperparam_tuning",
)

# Early stopping setup
early_stopping = EarlyStopping(monitor="val_loss", patience=5)

# Run the hyperparameter search
tuner.search(
    X_train,
    y_train,
    epochs=16,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping],
)

# Get the best hyperparameters and model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.get_best_models(num_models=1)[0]

# Show the best model summary
best_model.summary()

# Clean up memory after training
gc.collect()
tf.keras.backend.clear_session()

In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

model = build_model(best_hps)  # Create the model with best hyperparameters

history=model.fit(X_train, y_train,initial_epoch=16, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

In [ ]:
import matplotlib.pyplot as plt
history = history.history
# Plot 1: Training and Validation Loss
plt.subplot(1, 2, 1)  
plt.plot(history['val_loss'], color='red', label='Validation Loss')
plt.plot(history['loss'], color='blue', label='Training Loss')
plt.title('Validation and Training Loss ')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot 2: Training and Validation Accuracy
plt.subplot(1, 2, 2) 
plt.plot(history['val_accuracy'], color='red', label='Validation Accuracy')
plt.plot(history['accuracy'], color='blue', label='Training Accuracy')
plt.title('Validation and Training Accuracy ')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Adjust layout for better spacing
plt.tight_layout(pad=50.0)
plt.show()

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hps.values)

In [ ]:
y_pred = model.predict(X_test)
y_pred[0]

In [ ]:
fig, ax = plt.subplots(ncols = 5, nrows = 5, figsize = (12, 12))
ax = ax.flatten()

for i, img in enumerate(X_test[:25]):
    ax[i].axis('off')
    ax[i].imshow(img, cmap = "binary", interpolation = "nearest")
    y_pred_labels = np.argmax(y_pred, axis=1)
    ax[i].set_title(styles[y_pred_labels[i]], fontsize=9)
fig.suptitle('ArchiNet Test Dataset Predictions', fontsize = 16)

In [ ]:


report = classification_report(y_test, y_pred_labels, output_dict=True)

metrics = {
    "Accuracy": report['accuracy'],
    "Precision": report['macro avg']['precision'],
    "Recall": report['macro avg']['recall'],
    "F1-Score": report['macro avg']['f1-score']
}
metrics_df = pd.DataFrame(list(metrics.items()), columns=["Metric", "Score"])

plt.figure(figsize=(8, 5))
sns.barplot(data=metrics_df, x="Metric", y="Score", palette="viridis")

plt.title("Classification Metrics", fontsize=16)
plt.xlabel("Metric", fontsize=12)
plt.ylabel("Score", fontsize=12)
plt.ylim(0, 1.1)  # Metrics are typically between 0 and 1
plt.tight_layout()
plt.show()
